# 37-PSNR

离散余弦逆变换中如果不使用$8$作为系数，而是使用$4$作为系数的话，图像的画质会变差。来求输入图像和经过离散余弦逆变换之后的图像的峰值信噪比吧！再求出离散余弦逆变换的比特率吧！

峰值信噪比（Peak Signal to Noise Ratio）缩写为PSNR，用来表示信号最大可能功率和影响它的表示精度的破坏性噪声功率的比值，可以显示图像画质损失的程度。

峰值信噪比越大，表示画质损失越小。峰值信噪比通过下式定义。MAX表示图像点颜色的最大数值。如果取值范围是$[0,255]$的话，那么MAX的值就为255。MSE表示均方误差（Mean Squared Error），用来表示两个图像各个像素点之间差值平方和的平均数： $$ \text{PSNR}=10\ \log_{10}\ \frac{{v_{max}}^2}{\text{MSE}}\ \text{MSE}=\frac{\sum\limits_{y=0}^{H-1}\ \sum\limits_{x=0}^{W-1}\ [I_1(x,y)-I_2(x,y)]^2}{H\ W} $$ 如果我们进行$8\times8$的离散余弦变换，离散余弦逆变换的系数为$KtimesK$的话，比特率按下式定义： $$ \text{bit rate}=8\ \frac{K^2}{8^2} $$

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [1]:
# DCT hyoer-parameter
T = 8
K = 4
channel = 3

In [3]:
# DCT weight
def w(x, y, u, v):
    cu = 1.
    cv = 1.
    if u == 0:
        cu /= np.sqrt(2)
    if v == 0:
        cv /= np.sqrt(2)
    theta = np.pi / (2 * T)
    
    return (( 2 * cu * cv / T) * np.cos((2*x+1)*u*theta) * np.cos((2*y+1)*v*theta))

In [4]:
# DCT
def dct(img):
    H, W, _ = img.shape

    F = np.zeros((H, W, channel), dtype=np.float32)

    for c in range(channel):
        for yi in range(0, H, T):
            for xi in range(0, W, T):
                for v in range(T):
                    for u in range(T):
                        for y in range(T):
                            for x in range(T):
                                F[v+yi, u+xi, c] += img[y+yi, x+xi, c] * w(x,y,u,v)

    return F

In [5]:
# IDCT
def idct(F):
    H, W, _ = F.shape

    out = np.zeros((H, W, channel), dtype=np.float32)

    for c in range(channel):
        for yi in range(0, H, T):
            for xi in range(0, W, T):
                for y in range(T):
                    for x in range(T):
                        for v in range(K):
                            for u in range(K):
                                out[y+yi, x+xi, c] += F[v+yi, u+xi, c] * w(x,y,u,v)

    out = np.clip(out, 0, 255)
    out = np.round(out).astype(np.uint8)

    return out

In [6]:
# MSE
def MSE(img1, img2):
    H, W, _ = img1.shape
    mse = np.sum((img1 - img2) ** 2) / (H * W * channel)
    
    return mse

In [7]:
# PSNR
def PSNR(mse, vmax=255):
    
    return 10 * np.log10(vmax * vmax / mse)

In [8]:
# bitrate
def BITRATE():
    
    return 1. * T * K * K / T / T

In [9]:
img = cv2.imread('../picture/chans.png').astype(np.float32)

In [10]:
F = dct(img)

In [11]:
img_dct = idct(F)

In [12]:
# MSE
mse = MSE(img, img_dct)

# PSNR
psnr = PSNR(mse)

# bitrate
bitrate = BITRATE()

In [14]:
print("MSE:", mse)
print("PSNR:", psnr)
print("bitrate:", bitrate)

MSE: 294.2742106119792
PSNR: 23.443281574827807
bitrate: 2.0


In [15]:
cv2.imwrite('../picture/chan_result37_Discrete Cosine Transformation.jpg', img_dct)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_dct)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 38-JPEG 压缩——第二步：离散余弦变换+量化

量化离散余弦变换系数并使用离散余弦逆变换恢复。再比较变换前后图片的大小。

量化离散余弦变换系数是用于编码 JPEG 图像的技术。

量化即在对值在预定义的区间内舍入，其中floor、ceil、round等是类似的计算。

在 JPEG 图像中，根据下面所示的量化矩阵量化离散余弦变换系数。
该量化矩阵取自 JPEG 软件开发联合会组织颁布的标准量化表。在量化中，将8x 8的系数除以（量化矩阵） Q 并四舍五入。
之后然后再乘以 Q 。对于离散余弦逆变换，应使用所有系数。

In [16]:
# 量化
def quantization(F):
    H, W, _ = F.shape

    Q = np.array(((16, 11, 10, 16, 24, 40, 51, 61),
                (12, 12, 14, 19, 26, 58, 60, 55),
                (14, 13, 16, 24, 40, 57, 69, 56),
                (14, 17, 22, 29, 51, 87, 80, 62),
                (18, 22, 37, 56, 68, 109, 103, 77),
                (24, 35, 55, 64, 81, 104, 113, 92),
                (49, 64, 78, 87, 103, 121, 120, 101),
                (72, 92, 95, 98, 112, 100, 103, 99)), dtype=np.float32)

    for ys in range(0, H, T):
        for xs in range(0, W, T):
            for c in range(channel):
                F[ys: ys + T, xs: xs + T, c] =  np.round(F[ys: ys + T, xs: xs + T, c] / Q) * Q

    return F

In [17]:
F = quantization(F)

In [18]:
img_quan = idct(F)

In [19]:
# MSE
mse = MSE(img, img_quan)

# PSNR
psnr = PSNR(mse)

# bitrate
bitrate = BITRATE()

In [20]:
print("MSE:", mse)
print("PSNR:", psnr)
print("bitrate:", bitrate)

MSE: 296.60695393880206
PSNR: 23.40899032038506
bitrate: 2.0


In [21]:
cv2.imwrite('../picture/chan_result38_quantization.jpg', img_quan)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_quan)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 39-JPEG 压缩——第三步：YCbCr 色彩空间

在 YCbCr 色彩空间内，将 Y 乘以0.7以使对比度变暗。

YCbCr 色彩空间是用于将图像由表示亮度的 Y、表示蓝色色度Cb以及表示红色色度Cr表示的方法。

这用于 JPEG 转换。

使用下式从 RGB 转换到 YCbCr： $$ Y = 0.299 \ R + 0.5870 \ G + 0.114 \ B$$ $$Cb = -0.1687\ R - 0.3313 \ G + 0.5 \ B + 128$$ $$Cr = 0.5 \ R - 0.4187 \ G - 0.0813 \ B + 128 $$ 

使用下式从 YCbCr 转到 RGB： $$ R = Y + (Cr - 128) \ 1.402$$ $$G = Y - (Cb - 128) \ 0.3441 - (Cr - 128) \ 0.7139$$ 
$$B = Y + (Cb - 128) \ 1.7718$$

In [22]:
channel = 3

In [24]:
# BGR -> YCbCr
def BGR2YCbCr(img):
    H, W, C = img.shape

    ycbcr = np.zeros([H, W, 3], dtype=np.float32)

    ycbcr[..., 0] = 0.2990 * img[..., 2] + 0.5870 * img[..., 1] + 0.1140 * img[..., 0]
    ycbcr[..., 1] = -0.1687 * img[..., 2] - 0.3313 * img[..., 1] + 0.5 * img[..., 0] + 128.
    ycbcr[..., 2] = 0.5 * img[..., 2] - 0.4187 * img[..., 1] - 0.0813 * img[..., 0] + 128.

    return ycbcr

In [25]:
# YCbCr -> BGR
def YCbCr2BGR(ycbcr):
    
    H, W, C = img.shape
    
    bgr = np.zeros([H, W, channel], dtype=np.float32)
    bgr[..., 2] = ycbcr[..., 0] + (ycbcr[..., 1] - 128) * 1.402
    bgr[..., 1] = ycbcr[..., 0] - (ycbcr[..., 1] - 128) * 0.3411 - (ycbcr[..., 2] - 128) * 0.7139
    bgr[..., 0] = ycbcr[..., 0] + (ycbcr[..., 1] - 128) * 1.7718
    
    bgr = np.clip(bgr, 0, 255)
    bgr = bgr.astype(np.uint8)
    
    return bgr

In [26]:
ycbcr = BGR2YCbCr(img)

In [27]:
bgr = YCbCr2BGR(ycbcr)

In [28]:
cv2.imwrite('../picture/chan_result39_ycbcr.jpg', bgr)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()